In [5]:
import requests

def search_products(category):
    url = "http://127.0.0.1:5000/products/search-product"  # Replace with your actual URL
    params = {"searchTerm": category}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        # Handle errors based on response status code
        print(f"Error: {response.status_code}")
        return None
result = search_products("phone")
result

{'result': [{'categoryDetails': {'_key': 'best_seller',
    'categoryName': 'Best Sellers',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'},
   'productDetails': {'_key': 'prod_456',
    'productCategoryId': ['best_seller', 'free_delivery'],
    'productDescription': 'A high-quality smartphone with advanced features.',
    'productName': 'Smartphone Z',
    'productPicture': 'http://127.0.0.1:5000/static/images/iphone11promax.png',
    'reviews': [{'comment': 'Great phone!', 'userId': 'user_789'}],
    'variations': [{'availabilityQuantity': 10,
      'discountPrice': 0.0,
      'offerPrice': 999.0,
      'quantity': 1,
      'sellingPrice': 1099.0}]}},
  {'categoryDetails': {'_key': 'free_delivery',
    'categoryName': 'Free Delivery',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'},
   'productDetails': {'_key': 'prod_456',
    'productCategoryId': ['best_seller', 'free_delivery'],
    'productDescription': 'A high-quality smartphone wi

In [ ]:
{'result': [{
    'categoryDetails': {
        '_key': 'free_delivery',
        'categoryName': 'Free Delivery',
        'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'
    },
        'productDetails': {
            '_key': 'prod_456',
            'productDescription': 'A high-quality smartphone with advanced features.',
            'productName': 'Smartphone Z',
            'productPicture': 'http://127.0.0.1:5000/static/images/iphone11promax.png',
            'reviews': [
                    {
                    'comment': 'Great phone!', 'userId': 'user_789'
                    }
                ],
            'variations': [
                    {'availabilityQuantity': 10,
                    'discountPrice': 0.0,
                    'offerPrice': 999.0,
                    'quantity': 1,
                    'sellingPrice': 1099.0
                    }
                ]
            }
        }
    ]
 }